In [88]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder


In [104]:
train_data = pd.read_csv('train.csv',index_col=0)
test_data  = pd.read_csv('test.csv',index_col=0)

In [105]:
encoders = []

train_data = train_data.drop(columns = ['X', 'Y', 'Descript', 'Resolution'], errors = 'ignore')

for column in train_data.columns:
    labeler = LabelEncoder()
    column_string = column + 'Code'
    train_data[column_string] = labeler.fit_transform(train_data[column])
    encoders.append(labeler)
    train_data = train_data.drop(columns = column, errors = 'ignore')

y_train = train_data['CategoryCode']
X_train = train_data.drop(columns = 'CategoryCode', errors = 'ignore')

In [91]:
test_data = test_data.drop(columns = ['X', 'Y'])

for column in test_data.columns:
    column_string = column + 'Code'
    test_data[column_string] = LE.fit_transform(test_data[column])
    test_data = test_data.drop(columns = column)

X_test = test_data

In [92]:
params = {
    'objective': 'multi:softmax',
    'num_class': max(y_train)+1,
    'max_depth': 4,
    'eta': 0.1
}

dtrain = xgb.DMatrix(X_train, label=y_train)
# Train the XGBoost model
num_rounds = 100
xgb_model = xgb.train(params, dtrain, num_rounds)

In [93]:
X_test = X_test.drop(columns='DatesCode', errors = 'ignore')
#Create dummy test labels
y_test = pd.Series(data = 0, index = range(0, 884262))

dtest = xgb.DMatrix(X_test, label=y_test)
# Make predictions on the testing data
y_pred = xgb_model.predict(dtest)



In [113]:
ser = pd.Series(y_pred, name = 'CategoryCode')
ser = ser.astype('int64')
y_pred_final = encoders[0].inverse_transform(ser)
y_pred_final = pd.Series(y_pred_final , name='Category')
y_pred_final = pd.get_dummies(y_pred_final, dtype = int)

print(y_pred_final)

        ASSAULT  DISORDERLY CONDUCT  DRUG/NARCOTIC  LARCENY/THEFT  \
0             0                   0              0              1   
1             0                   0              0              0   
2             0                   0              0              1   
3             1                   0              0              0   
4             1                   0              0              0   
...         ...                 ...            ...            ...   
884257        0                   0              0              1   
884258        0                   0              0              1   
884259        0                   0              0              1   
884260        0                   0              0              0   
884261        0                   0              0              1   

        MISSING PERSON  NON-CRIMINAL  OTHER OFFENSES  PROSTITUTION  RUNAWAY  \
0                    0             0               0             0        0   
1            

In [114]:
y_pred_final.to_csv('submission.csv')